## Paso 1

Cargamos la data del archivo ipynb para utilizarlo en la lista data, de la extraction de los features vectors.

In [1]:
import os
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


# Especifica la ruta a tu carpeta
folder_path = '/Users/pierre/Documents/6_ciclo/machineLearning/project3/features/r21d/r2plus1d_18_16_kinetics/'

# Lista para almacenar los datos cargados
data_list = []

for filename in os.listdir(folder_path):
    if filename.endswith('.npy'):
        file_path = os.path.join(folder_path, filename)
        data = np.load(file_path)
        data_list.append(data)

## Paso 2

Calculamos la varianza explicada para saber cuantos datos usamos, para obtener el mejor n_components.

In [8]:
from sklearn.decomposition import FactorAnalysis

explained_variances = []
optimal_n_components_list = []
scaler = StandardScaler()
total_pca_reconstruction_error = 0
total_svd_reconstruction_error = 0
total_fa_reconstruction_error = 0
num_files = len(data_list)
svd_components_save = []
pca_components_save = []
for i, data in enumerate(data_list):
    print(f'Datos del archivo {i+1}:')
    # Normalización de los datos
    X_scaled = scaler.fit_transform(data)
    
    # Calcula la matriz de covarianza
    cov_matrix = np.cov(X_scaled.T)
    
    # Calcula los autovalores y autovectores de la matriz de covarianza
    eigenvalues, _ = np.linalg.eig(cov_matrix)
    
    # Calcula la varianza explicada
    total_variance = np.sum(eigenvalues)
    explained_variance_ratio = eigenvalues / total_variance
    explained_variances.append(np.sum(explained_variance_ratio))
    
    print(f'Varianza explicada del archivo {i+1}: {explained_variances[-1]}')
    
    # Determina el número óptimo de componentes
    cumulative_variance_ratio = np.cumsum(explained_variance_ratio)
    optimal_n_components = np.argmax(cumulative_variance_ratio >= 0.98) + 1
    optimal_n_components_list.append(optimal_n_components)
    print(f'Número óptimo de componentes: {optimal_n_components}')
    
    # PCA
    pca = PCA(n_components=optimal_n_components)
    pca_components = pca.fit_transform(X_scaled)
    
    # SVD
    svd = TruncatedSVD(n_components=optimal_n_components)  
    svd_components = svd.fit_transform(X_scaled)
    
    # FA
    fa = FactorAnalysis(n_components=optimal_n_components)
    fa_components = fa.fit_transform(X_scaled)
    
    # Error de reconstrucción PCA
    X_pca_reconstructed = pca.inverse_transform(pca_components)
    pca_reconstruction_error = np.mean((X_scaled - X_pca_reconstructed) ** 2)
    total_pca_reconstruction_error += pca_reconstruction_error
    
    # Error de reconstrucción SVD
    X_svd_reconstructed = svd.inverse_transform(svd_components)
    svd_reconstruction_error = np.mean((X_scaled - X_svd_reconstructed) ** 2)
    total_svd_reconstruction_error += svd_reconstruction_error
    
    # Error de reconstrucción FA
    X_fa_reconstructed = np.dot(fa_components, fa.components_) + fa.mean_
    fa_reconstruction_error = np.mean((X_scaled - X_fa_reconstructed) ** 2)
    total_fa_reconstruction_error += fa_reconstruction_error
    
    svd_components_save.append(svd_components)
    pca_components_save.append(pca_components)
    # Resultados
    print(f'Error de reconstrucción por PCA: {pca_reconstruction_error}')
    print(f'Error de reconstrucción por SVD: {svd_reconstruction_error}')
    print(f'Error de reconstrucción por FA: {fa_reconstruction_error}')
    
print("Varianzas explicadas:", explained_variances)
print("Números óptimos de componentes:", optimal_n_components_list)

average_pca_reconstruction_error = total_pca_reconstruction_error / num_files
average_svd_reconstruction_error = total_svd_reconstruction_error / num_files
average_fa_reconstruction_error = total_fa_reconstruction_error / num_files

print(f'Error de reconstrucción promedio por PCA: {average_pca_reconstruction_error}')
print(f'Error de reconstrucción promedio por SVD: {average_svd_reconstruction_error}')
print(f'Error de reconstrucción promedio por FA: {average_fa_reconstruction_error}')




Datos del archivo 1:
Varianza explicada del archivo 1: (0.9999999999999999-5.007417855406813e-33j)
Número óptimo de componentes: 15
Error de reconstrucción por PCA: 0.015309481271750865
Error de reconstrucción por SVD: 0.015309481271750861
Error de reconstrucción por FA: 0.0162556979877029
Datos del archivo 2:
Varianza explicada del archivo 2: (1-9.244463733058732e-33j)
Número óptimo de componentes: 13
Error de reconstrucción por PCA: 0.013337679541800878
Error de reconstrucción por SVD: 0.013337679541800877
Error de reconstrucción por FA: 0.013506147929259813
Datos del archivo 3:
Varianza explicada del archivo 3: (1+1.617781153285278e-32j)
Número óptimo de componentes: 15
Error de reconstrucción por PCA: 0.016794798877843922
Error de reconstrucción por SVD: 0.01679479887784392
Error de reconstrucción por FA: 0.0175552603296211
Datos del archivo 4:
Varianza explicada del archivo 4: (0.9999999999999998-2.8888949165808538e-33j)
Número óptimo de componentes: 16
Error de reconstrucción por

In [7]:
print(svd_components_save)

[array([[ 3.07856425e+00, -7.23795031e+00, -1.09517288e+01,
        -9.65723202e+00,  2.77246448e+00,  3.28473041e+00,
         9.62708384e-01,  1.92635756e+00,  4.43191060e+00,
         6.05872853e+00, -4.40777549e+00, -6.02091576e+00,
        -1.98765473e+00,  2.48726716e+00,  2.21837230e+00],
       [ 1.04810148e+00,  4.30319557e-01, -2.09543295e+01,
        -6.43308443e+00,  4.53249240e+00,  5.13508859e+00,
         5.09773532e+00, -1.30329653e+00,  1.59154819e+00,
        -2.84253970e+00,  6.27153399e+00,  3.39966529e+00,
         5.81532846e-01, -1.66527562e+00, -1.52802547e+00],
       [ 8.75151776e+00,  1.21217745e+01, -1.12189300e+01,
         6.71454945e+00, -2.18345800e-01, -6.43643827e+00,
        -3.73762371e+00, -1.13316290e+01, -6.14234272e+00,
         8.34376711e-01, -2.92370009e+00, -1.17718222e+00,
        -7.14705597e-01,  2.07589339e-01, -1.74425045e-01],
       [ 8.12132904e+00,  1.24871069e+01, -7.30384334e+00,
         1.56897941e+01, -4.05716528e+00,  1.0100420

In [9]:
print(pca_components_save)

[array([[ 3.07856425e+00, -7.23795031e+00, -1.09517288e+01,
        -9.65723202e+00,  2.77246448e+00,  3.28473041e+00,
         9.62708384e-01,  1.92635756e+00,  4.43191060e+00,
         6.05872853e+00, -4.40777549e+00, -6.02091576e+00,
        -1.98765473e+00,  2.48726716e+00,  2.21837230e+00],
       [ 1.04810148e+00,  4.30319557e-01, -2.09543295e+01,
        -6.43308443e+00,  4.53249240e+00,  5.13508859e+00,
         5.09773532e+00, -1.30329653e+00,  1.59154819e+00,
        -2.84253970e+00,  6.27153399e+00,  3.39966529e+00,
         5.81532846e-01, -1.66527562e+00, -1.52802547e+00],
       [ 8.75151776e+00,  1.21217745e+01, -1.12189300e+01,
         6.71454945e+00, -2.18345800e-01, -6.43643827e+00,
        -3.73762371e+00, -1.13316290e+01, -6.14234272e+00,
         8.34376711e-01, -2.92370009e+00, -1.17718222e+00,
        -7.14705597e-01,  2.07589339e-01, -1.74425045e-01],
       [ 8.12132904e+00,  1.24871069e+01, -7.30384334e+00,
         1.56897941e+01, -4.05716528e+00,  1.0100420